## Árbol de decisión

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que reduzca en mayor medida la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Varianza de cada rama ponderada por el número de gente de cada una de las ramas.

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Impureza de cada rama ponderada por el número de personas en cada una de ellas.

---

## Random Forest

Utilizas la técnica de bootsrap para hacer muestreos aleatorios de tus datos, sobre los cuales entrenas tu modelo de árbol de decisión. Cada uno de estos modelos realiza sus predicciones, y el promedio de estas sería la predicción utilizada por el algoritmo de Random Forest.

---

## Gradient Boosting

Proceso iterativo el cual mejora los errores cometidos en pasos anteriores. Inicia con una predicción a la cual le calcula el error, después se entrena un nuevo árbol sobre estos errores, con esto se actualiza la predicción del modelo total. Esto se hace a través de un factor de aprendizaje que pondera las predicciones anteriores y se va repitiendo el proceso.

Cada vez usa el error anterior para en cada iteración ir haciendo esté más pequeño. La predicción del árbol anterior es el promedio de los residuales obtenidos en cada una de las hojas. Modelo nuevo = modelo anteriror + v (nuevo árbol). Para clasificación se utilizan las log-odds.

---

## XGBoost

Sumas árboles óptimos.

XGBoost construye un modelo final a través de una suma de árboles.

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Donde:
- $\hat{y}_i^{(t)}$ es la predicción.
- $f_t(x_i)$ es el nuevo árbol.

En cada una de las iteraciones se busca minimizar la contribución marginal a la función de pérdida total.

La función de pérdida se compone de dos partes, una que son los árboles anterirores y otra parte que representa el árbol actual. La parte de los árboles anteriores se usa cmo constante (ya esta dado) y se optimiza lo que aporta el nuevo árbol, llegando a:

$$
\mathcal{L}^{(t)} =
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \sum_{k=1}^{t} \Omega(f_k)
$$

$\Omega(f_k)$ es un factor de regularización el cual castiga árboles más complejos.

Como la función de pérdida es compleja, se utiliza las series de Taylor para poder aproximar la función de pérdida cerca de las predicciones. La serie de Taylor de segundo orden permite aproximar la función de pérdida con una forma cuadrática, la cual es más sencilla de optmizar y velve más eficiente el proceso. Utilizando esta expansión, el gradiente y el hessiano la función de pérdida queda de la siguiente forma:

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Con ello la optimización es un problema cuadrático más sencillo de optimizar.

Cada nuevo árbol $f_t$ asigna un valor constante $w_j$. Entonces:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Con:
- $G_j = \sum_{i \in R_j} g_i$
- $H_j = \sum_{i \in R_j} h_i$

Después minimizas la función de pérdida con respecto de $w_j$ para obtener el output óptimo por hoja:

$$
\textbf{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$

Con estos valores óptimos calculas el similarity score para saber cuanto mejora la pérdida en cada una de loas hojas (nos dice la calidad de la hoja):

$$
\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}
$$

Cuando calculas el similarity score lo utilizas para obtener el gain, el cual nos dice si vale la pena hacer un split (aumentar la complejidad del árbol).

$$
\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma
$$

La predicción final se obtiene sumando todos los árboles:

$$
\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)
$$

---

## Comparativa de modelos

La estructura del árbol:

* XGBoost produce árboles más simétricos y balanceados.

* LightGBM produce árboles más profundos y desbalanceados si no se controla.

La precisión y riesgo de overfitting:

* Leaf-wise (LightGBM) puede encontrar mejores divisiones, pero se sobreajusta más fácil.

* Level-wise (XGBoost) es más estable, pero a veces menos preciso.

| Característica              | **XGBoost**                                                | **LightGBM**                                               | **CatBoost**                                                  |
|-----------------------------|------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|
| **Velocidad**               | Rápido, pero más lento que LightGBM y CatBoost             | 🔥 Muy rápido gracias a histogramas y leaf-wise growth      | Rápido, aunque un poco más lento que LightGBM                  |
| **Precisión**               | Alta                                                       | Alta, a veces mejor con buen tuning                         | Muy alta, especialmente con categóricas                        |
| **Variables categóricas**   | ❌ No las maneja (requiere encoding manual)                | ❌ No las maneja (requiere encoding manual)                 | ✅ Soporte nativo + regularización secuencial                  |
| **Uso de memoria**          | Moderado                                                   | ✅ Muy eficiente (binning)                                   | Similar a XGBoost                                              |
| **Manejo de missing values**| ✅ Automático                                               | ✅ Automático                                                | ✅ Automático                                                   |
| **Soporte GPU**             | ✅ Sí (bastante estable)                                   | ✅ Sí (muy rápido)                                           | ✅ Sí (algo más limitado)                                      |
| **Instalación**             | Fácil (`pip install xgboost`)                             | Fácil (`pip install lightgbm`)                              | Un poco más pesada (`pip install catboost`)                   |
| **Documentación**           | Excelente                                                  | Buena                                                       | Muy buena                                                     |
| **Interacción con sklearn** | Muy buena                                                  | Muy buena                                                   | Muy buena                                                     |
| **Tolerancia al orden**     | ✅ Neutral                                                  | ✅ Neutral                                                   | ⚠️ Sensible (por codificación secuencial)                      |

Lightgbm funciona a través de bins, mientras que catboost deja las mismas ramas, lo cual hace una predicción más veloz.

Promedio de tu variable de forma secuancial (categorical encoding). Usa el promedio conocido hasta ese momento.